In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import pyvista as pv

ModuleNotFoundError: No module named 'pyvista'

In [ ]:
# Replace 'full/path/to/your/merged_df.csv' with the actual full path to your CSV file
file_path = r'C:\Users\betebari\Documents\C2VSim_Texture\AEM supporting data\merged_file.csv'

# Read the CSV file into a pandas DataFrame
merged_df = pd.read_csv(file_path)

In [ ]:
# Initialize an empty DataFrame before the loop
merged_df = pd.DataFrame()

# File paths for your CSV files
file_paths = ['AEM_WELL_LITHOLOGY_csv_WO4_20230831.csv', 'AEM_WELL_LITHOLOGY_csv_WO5_20230125.csv', 'AEM_WELL_LITHOLOGY_csv_WO6_20230103.csv', 'AEM_WELL_LITHOLOGY_csv_WO7_20230327.csv']

for file_path in file_paths:
    # Specify the encoding when reading the CSV files
    df = pd.read_csv(file_path, encoding='latin1')
    merged_df = merged_df.append(df, ignore_index=True)
    
# merged_df.to_csv('merged_file.csv', index=False)
print(merged_df)

In [ ]:
# Subtract Column2 from Column1
merged_df['depth'] = merged_df['GROUND_SURFACE_ELEVATION_m'] - merged_df['LITH_TOP_DEPTH_m']

In [ ]:
import matplotlib.pyplot as plt

# Visualize the well samples using a scatter plot
fig = plt.figure(figsize=(10, 8))
ax = fig.add_subplot(111, projection='3d')

# Map classification to color
class_to_color = {
    'SC-SM': 'blue',
    'GW/GP': 'green',
    'ML/MH': 'yellow',
    'CH/CL': 'lightcoral',
    'SW/SP': 'lightblue',
    'SC': 'purple',
    'OL/OH': 'orange',
    'GC': 'pink',
    'Rock': 'gray',
    'SM': 'brown'
}

for classification, color in class_to_color.items():
    class_df = merged_df[merged_df['USCS_SYMBOL'] == classification]
    ax.scatter(
        class_df['X_CATEALEALBERS_m'],
        class_df['Y_CATEALEALBERS_m'],
        class_df['depth'],
        c=color,
        label=classification,
        alpha=0.1
    )

# Customize plot
ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.set_zlabel('Z')
ax.set_title('Well Samples Classification')
ax.legend()

# Add transparency and set view rotation
ax.view_init(elev=20, azim=10)  # Set the elevation and azimuthal angles for rotation

# Show the plot
plt.show()

In [ ]:
# Export data to Paraview format (VTK)
vtk_file_path = 'well_samples.vtk'
merged_df.to_parquet(vtk_file_path, index=False)

In [ ]:
# Load your mesh (replace 'path/to/your/mesh.vtk' with your actual file path)
mesh_file_path = 'well_samples.vtk'
mesh = pv.read(mesh_file_path)

# Load your soil data (replace 'path/to/your/soil_data.csv' with your actual file path)
soil_data_file_path = 'path/to/your/soil_data.csv'
soil_data = pd.read_csv(soil_data_file_path)

# Assuming your soil data has columns 'X', 'Y', 'Z', and 'Depth'
# and 'Depth' represents the depth of the soil at each point

# Create a PyVista mesh from the soil data
soil_mesh = pv.PolyData(soil_data[['X', 'Y', 'Z']].values)

# Add soil data to each node based on depth intervals (each 100 feet)
depth_intervals = range(0, int(soil_data['Depth'].max()) + 100, 100)

for depth_start, depth_end in zip(depth_intervals, depth_intervals[1:]):
    # Extract nodes within the depth interval
    selected_nodes = mesh.find_points((depth_start, depth_end), 'Z')

    # Assign soil data to the selected nodes
    selected_soil_data = soil_mesh.select_points(points=selected_nodes, invert=False)
    mesh.point_arrays[f'SoilData_{depth_start}-{depth_end}'] = selected_soil_data['Depth']

# Save the updated mesh with soil data
mesh_with_soil_data_file_path = 'path/to/your/mesh_with_soil_data.vtk'
mesh.save(mesh_with_soil_data_file_path)